# Load A/C & Ventilation Data

We scraped the following link for downloadable Excel files for both A/C and ventilation data: https://www.nycenet.edu/roomassessment?code=M001. Then we consolidated those downloaded Excel files into a single CSV.

Interestingly, the IBO "barriers to schools" report uses a different A/C dataset from a report made for City Council by DOE. Find it on [this page](https://council.nyc.gov/budget/fy2021/) for the 2021 City Council budget under the "Terms and Conditions" section in the hyperlink for "Department of Education – Air Conditioning Report (1 of 2) (XLSX)". This data only exists for FY2021 and prior. Theoretically, the data we pulled above is more recent, but the catch is that the DOE AC report to city council was wrapped up a little better (i.e. it had already done the math of sqft with A/C by building code) and had extra stats like cost estimated for remaining A/C buildout.

I explored joining our A/C data onto the [buildings space data](https://data.cityofnewyork.us/Education/DOE-Building-Space-Usage/wavz-fkw8/about_data) from Open Data NYC to get a pct area figure of square feet covered by A/C. But that is proving to be messy because the joins don't work very cleanly. So, instead I am going to just do a proportion of regular classrooms with A/C. That follows the methodology from DOE's reports on A/C to City Council, where they focused on instructional rooms rather than gyms, auditoriums, cafeterias, libraries, etc.

I think the main metric we'll put in the dashboard is proportion of classrooms with A/C, but I want to add in to that data the proportion of all the other room types so that we can see when a building is truly 100% and when it's not. Also it matters if the gym doesn't have A/C.

In [ ]:
import pandas as pd

ac_df = pd.read_csv('../data/raw_data/DOE/Ventilation/ac_dataset.csv')
ventilation_df = pd.read_csv('../data/raw_data/DOE/Ventilation/ra_dataset.csv')
# Drop unneeded column
ac_df = ac_df.drop(columns=['index in original file'])
ventilation_df = ventilation_df.drop(columns=['index in original file'])
# Drop duplicates after removing the index column
ac_df = ac_df.drop_duplicates()
ventilation_df = ventilation_df.drop_duplicates()
# Clean up join keys
ac_df['BuildingCode'] = ac_df['BuildingCode'].str.strip().str.upper()
ac_df['Room'] = ac_df['Room'].str.strip().str.upper()
ventilation_df['BuildingCode'] = ventilation_df['BuildingCode'].str.strip().str.upper()
ventilation_df['Room'] = ventilation_df['Room'].str.strip().str.upper()

# Process A/C Data

In [ ]:
ac_df['Primary Usage'].value_counts()

In [ ]:
ac_df['Room Status'].value_counts()

We're going to use "Operational" as the primary metric of whether A/C is present. My understanding is that this will be data that is current as of the time we scraped the data: 10/26/2025. So A/C in repair is a recent metric. 

If we need a more permanent metric, we might just use "No A/C" instead to avoid the repairs making things confusing. If we use "No A/C", it makes it seem like you could 1-n to get the affirmative (i.e. x% *do* have A/C), but that's not true bc of the repairs category. So idk maybe people will find the operational thing confusing, but just trying to work around this repair category.

In [ ]:
# Less conservative treatment of A/C: Calculate percentage of "Operational" A/C for each BuildingCode and Primary Usage combination
# This will include all buildings, even those with 0% "Operational"
ac_operational_summary = ac_df.groupby(['BuildingCode', 'Primary Usage']).apply(
    lambda x: (x['Room Status'] == 'Operational').sum() / len(x)
).unstack(fill_value=0)
# Rename columns to include "_Operational" suffix
ac_operational_summary.columns = [f"{col}_Op_AC".replace(' ', '_') for col in ac_operational_summary.columns]
# Reset index to get BuildingCode as a regular column
ac_operational_summary = ac_operational_summary.reset_index()
# Shorten column names for shapefile 10 char limit
ac_operational_summary.columns = ac_operational_summary.columns.str.replace(
    'AUDITORIUM', 'AUD').str.replace(
        'CAFETERIA', 'CAF').str.replace(
            'REGULAR_CLASSROOM', 'CLS').str.replace(
                'GYMNASIUM', 'GYM').str.replace(
                    'LIBRARY', 'LIB').str.replace(
                        'MULTIPURPOSE', 'MPR')

# Filter for buildings with <100% classrooms with operational A/C
ac_operational_summary[ac_operational_summary['CLS_Op_AC'] < 1.0]


In [ ]:
# Most conservative treatment of A/C: Calculate percentage of "No" A/C" for each BuildingCode and Primary Usage combination
# This will include all buildings, even those with 0% "No A/C"
no_ac_summary = ac_df.groupby(['BuildingCode', 'Primary Usage']).apply(
    lambda x: (x['Room Status'] == 'No A/C').sum() / len(x)
).unstack(fill_value=0)
# Rename columns to include "_No_AC" suffix
no_ac_summary.columns = [f"{col}_No_AC".replace(' ', '_') for col in no_ac_summary.columns]
# Reset index to get BuildingCode as a regular column
no_ac_summary = no_ac_summary.reset_index()
# Shorten column names for shapefile 10 char limit
no_ac_summary.columns = no_ac_summary.columns.str.replace(
    'AUDITORIUM', 'AUD').str.replace(
        'CAFETERIA', 'CAF').str.replace(
            'REGULAR_CLASSROOM', 'CLS').str.replace(
                'GYMNASIUM', 'GYM').str.replace(
                    'LIBRARY', 'LIB').str.replace(
                        'MULTIPURPOSE', 'MPR')

# Filter for buildings with >0% classrooms with No A/C
no_ac_summary[no_ac_summary['CLS_No_AC'] > 0]

In [ ]:
# Show top 20 buildings with highest percentage of classrooms with No A/C
no_ac_summary.sort_values('CLS_No_AC', ascending=False).head(20)

# Export A/C Data

Exporting both approaches to defining A/C status: "no A/C at all" and "no *working* A/C at the time of data scraping"

In [ ]:
no_ac_summary.to_csv('../data/processed_data/no_ac_summary.csv', index=False)
ac_operational_summary.to_csv('../data/processed_data/ac_operational_summary.csv', index=False)

# Process Ventilation Data

According to the webpages we scraped this data from (e.g. [here](https://www.nycenet.edu/roomassessment?code=K170)):
>Ventilation = any way for fresh air to enter a room including operational windows, mechanical ventilation, or a combination of both.


It's a mystery to me what the `AtLeast` column is in this data. You'd think it would be something like "there is at least one ventilation method". But the below assertions don't bear that out.

For now, I'm going to chock this up to a mistake in the data and DIY the boolean field for whether ventilation is present, based on the definition provided above.

### This data is kind of confusing

We can see that the `AtLeast` column is never "Yes" when none of the ventilation methods exist, which makes sense

In [ ]:
# Show that `AtLeast` column is never "Yes" when none of the ventilation methods exist
assert ventilation_df[
    (ventilation_df['AtLeast']=='Yes') & 
    (ventilation_df['Windows']=='No') & 
    (ventilation_df['Exhaust Fan']=="Doesn't Exist") & 
    (ventilation_df['Supply Fan']=="Doesn't Exist")
    ].empty

But the inverse is perplexing: we have situations where `AtLeast` is still "No" even when Windows and/or fully-functiong fans are present. That shouldn't happen if `AtLeast` corresponds to the above definition.

In [ ]:
# Case 1: AtLeast is "No" even when Windows AND full-functioning fans are present (i.e. both supply and exhaust are operational)
ventilation_df[
        (ventilation_df['AtLeast']=='No') & 
        (ventilation_df['Windows']=='Yes') &
        (ventilation_df['Exhaust Fan']=="Operational") &
        (ventilation_df['Supply Fan']=="Operational")
    ]

In [ ]:
# Case 2: AtLeast is "No" when either Windows or full-functioning fans are present (i.e. both supply and exhaust are operational)
ventilation_df[
    (
        (ventilation_df['AtLeast']=='No') & 
        (
            (ventilation_df['Windows']=='Yes') |
            (
                (ventilation_df['Exhaust Fan']=="Operational") &
                (ventilation_df['Supply Fan']=="Operational")
            )
        )
    )
    ]

### Screw it, we're going to just work around the weirdness

So, we're going to DIY the boolean column for "no ventilation", using the above definition from the NYC Schools website: "no ventilation" means either there is no window or there is no supply/exhaust fan. 

In [ ]:
sorted(ventilation_df['Exhaust Fan'].unique())

In [ ]:
sorted(ventilation_df['Supply Fan'].unique())

If we were to use the most methodologically simple and conservative option (i.e. treat "no ventilation" as no existing infrastructure for ventilation at all), there are NO classrooms in this data that meet that definition.

In [ ]:
# Most conservative definition of "no ventilation": no windows AND no existing supply/exhaust fans
ventilation_df[
        (ventilation_df['Primary Usage'].str.upper().str.contains('CLASSROOM')) &
        (ventilation_df['Windows']=='No') &
        (ventilation_df['Exhaust Fan']=="Doesn't Exist") &
        (ventilation_df['Supply Fan']=="Doesn't Exist")
    ]['BuildingCode'].nunique()

Instead, we're going to focus on the existence of BOTH a supply and exhaust fan in a classroom. The rationale here is that with increased wildfire smoke polluting our air, students need fully functioning mechanical ventilation in every classroom, not just windows (which are actually a hazard on smokey days). With GHS renovations, mechanical ventilation will also be necessary so that  HEPA filters can be used to purify air.

In [ ]:
# Show how many school builings have at least one classroom that's missing either a supply or exhaust fan
ventilation_df[
        (ventilation_df['Primary Usage'].str.upper().str.contains('CLASSROOM')) &
        (
            (ventilation_df['Exhaust Fan']=="Doesn't Exist") |
            (ventilation_df['Supply Fan']=="Doesn't Exist")
        )
    ]['BuildingCode'].nunique()

In [ ]:
# Create flag for these classrooms missing at least one element of mechanical ventilation
ventilation_df['missing_mech_ventilation'] = (
        (
            (ventilation_df['Exhaust Fan']=="Doesn't Exist") |
            (ventilation_df['Supply Fan']=="Doesn't Exist")
        )
)

In [ ]:
ventilation_df[ventilation_df['missing_mech_ventilation']]['Primary Usage'].value_counts()

In [ ]:
no_vent_summary = ventilation_df.groupby(['BuildingCode', 'Primary Usage']).apply(
    lambda x: (x['missing_mech_ventilation']).sum() / len(x)
).unstack(fill_value=0)
# Rename columns to include "_No_VT" suffix
no_vent_summary.columns = [f"{col.replace(' ', '_').upper()}_No_VT" for col in no_vent_summary.columns]
# Reset index to get BuildingCode as a regular column
no_vent_summary = no_vent_summary.reset_index()
# Shorten column names for shapefile 10 char limit
no_vent_summary.columns = no_vent_summary.columns.str.replace(
    'AUDITORIUM', 'AUD').str.replace(
        'CAFETERIA', 'CAF').str.replace(
            'STUDENT_CLASSROOM', 'CLS').str.replace(
                'GYMNASIUM', 'GYM').str.replace(
                    'BATHROOM', 'BTH').str.replace(
                        'MULTI-PURPOSE_ROOM', 'MPR').str.replace(
                            'STAFF_OFFICE', 'OFF').str.replace(
                                'KITCHEN', 'KIT').str.replace(
                                    'STUDENT_STAFF_SPACE_(LIBRARY,_SHOP,_SCIENCE_LAB,_MEDIA_CENTER_ETC.)', 'LAB')

In [ ]:
# Drop columns we won't use
no_vent_summary = no_vent_summary.drop(columns=[
    'BUILDING_SUPPORT_SPACE_No_VT',
    'CLOSET/STORAGE_ROOM_No_VT',
    'INACCESSIBLE_No_VT',
    'LOCKER_ROOM_No_VT'
])

In [ ]:
# Filter for buildings with >0% classrooms with Missing mechanical ventilation
no_vent_summary[no_vent_summary['CLS_No_VT'] > 0]

## Export Ventilation Data

In [ ]:
no_vent_summary.to_csv('../data/processed_data/missing_ventilation_summary.csv', index=False)